# TDA@YSDA
## Seminar 6

In the following we consider examples from

A Topology Layer for Machine Learning  
Brüel-Gabrielsson et al. _International Conference on Artificial Intelligence and Statistics (2020)_


https://github.com/bruel-gabrielsson/TopologyLayer.

In [ ]:
!pip install git+https://github.com/bruel-gabrielsson/TopologyLayer.git

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
from topologylayer.nn import AlphaLayer, BarcodePolyFeature
from topologylayer.nn import LevelSetLayer2D, SumBarcodeLengths, PartialSumBarcodeLengths

### Point cloud optimization

In [ ]:
# random pointcloud
data = np.random.rand(100, 2)

In [ ]:
# optimization to increase size of holes
layer = AlphaLayer(maxdim=1)
x = torch.autograd.Variable(torch.tensor(data).type(torch.float), requires_grad=True)
f1 = BarcodePolyFeature(1,2,0)
optimizer = torch.optim.Adam([x], lr=1e-2)
for i in range(100):
    optimizer.zero_grad()
    loss = -f1(layer(x))
    loss.backward()
    optimizer.step()
    
x = x.detach().numpy()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 5.75))
ax[0].set_title("Before")
ax[1].set_title("After")
ax[0].scatter(data[:,0], data[:,1])
ax[1].scatter(x[:,0], x[:,1])
plt.show()

### Level set optimization

In [ ]:
class TopLoss(nn.Module):
    def __init__(self, size):
        super(TopLoss, self).__init__()
        self.pdfn = LevelSetLayer2D(size=size,  sublevel=False)
        self.topfn = PartialSumBarcodeLengths(dim=1, skip=1) # penalize more than 1 hole
        self.topfn2 = SumBarcodeLengths(dim=0) # penalize more than 1 max

    def forward(self, beta):
        dgminfo = self.pdfn(beta)
        return self.topfn(dgminfo) + self.topfn2(dgminfo)

In [ ]:
# generate circle on grid
n = 50
def circlefn(i, j, n):
    r = np.sqrt((i - n/2.)**2 + (j - n/2.)**2)
    return np.exp(-(r - n/3.)**2/(n*2))


def gen_circle(n):
    beta = np.empty((n,n))
    for i in range(n):
        for j in range(n):
            beta[i,j] = circlefn(i,j,n)
    return beta

beta = gen_circle(n)

In [ ]:
m = 1500
X = np.random.randn(m, n**2)
y = X.dot(beta.flatten()) + 0.05*np.random.randn(m)
beta_ols = (np.linalg.lstsq(X, y, rcond=None)[0]).reshape(n,n)

In [ ]:
tloss = TopLoss((50,50)) # topology penalty
dloss = nn.MSELoss() # data loss

beta_t = torch.autograd.Variable(torch.tensor(beta_ols).type(torch.float), requires_grad=True)
X_t = torch.tensor(X, dtype=torch.float, requires_grad=False)
y_t = torch.tensor(y, dtype=torch.float, requires_grad=False)
optimizer = torch.optim.Adam([beta_t], lr=1e-2)
for i in range(500):
    optimizer.zero_grad()
    tlossi = tloss(beta_t)
    dlossi = dloss(y_t, torch.matmul(X_t, beta_t.view(-1)))
    loss = 0.1*tlossi + dlossi
    loss.backward()
    optimizer.step()
    if (i % 10 == 0):
        print(i, tlossi.item(), dlossi.item())
        
beta_est = beta_t.detach().numpy()

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(16,5))
ax[0].imshow(beta)
ax[0].set_title("Truth")
ax[1].imshow(beta_ols)
ax[1].set_title("OLS")
ax[2].imshow(beta_est)
ax[2].set_title("Topology Regularization")
for i in range(3):
    ax[i].set_yticklabels([])
    ax[i].set_xticklabels([])
    ax[i].tick_params(bottom=False, left=False)

### Quantifying point clouds (2 points)

#### Geometry score

Consider sklearn `digits` dataset. Consider a dimensionality reduction method of your choice - PCA, Isomap, autoencoder to reduce the dimensionality of data to the lower-dimensional space. Compute and plot the $\mathrm{RLT}^1$ distributions for the data in the original $\mathcal{X}$ and low-dimensional $\mathcal{Z}$ spaces along with Geometry Score $\mathrm{GS}(X, Z)$. Plot the graph of $\mathrm{GS}(X, Z)$ against various choices of the dimension of $\mathcal{Z}$, and, if compatible with your method, plot the graph of $\mathrm{GS}(X, Z)$ against the optimization step.

In [ ]:
# your code here

#### Manifold topology divergence

Implement the same for $\mathrm{MTop-Div}(X, Z)$.

In [ ]:
# your code here